In [112]:

# bundle = {"application/vnd.vega.v5+json": schema}
# print("", end=None)
# IPython.display.display(bundle, raw=True)





In [113]:
import networkx as nx
import numpy as np
import pandas as pd
from pyciemss.integration_utils.result_processing import convert_to_output_format
from pyciemss.visuals import  vega, plots
from pathlib import Path
import os
import json 

from IPython.display import display

In [114]:
import json
import random
from itertools import chain
from pathlib import Path

import networkx as nx
import numpy as np
import pandas as pd
import pytest

import pyciemss
from pyciemss.integration_utils.result_processing import convert_to_output_format
from pyciemss.visuals import plots, vega



In [115]:
def distributions():
    model_1_path = (
            "https://raw.githubusercontent.com/DARPA-ASKEM/simulation-integration"
            "/main/data/models/SEIRHD_NPI_Type1_petrinet.json"
    )
    start_time = 0.0
    end_time = 100.0
    logging_step_size = 1
    num_samples = 30
    sample = pyciemss.sample(
        model_1_path,
        end_time,
        logging_step_size,
        num_samples,
        start_time=start_time,
        solver_method="euler",
    )["unprocessed_result"]

    for e in sample.values():
        if len(e.shape) > 1:
            num_timepoints = e.shape[1]

    return convert_to_output_format(
        sample,
        timepoints=np.linspace(start_time, end_time, num_timepoints),
        time_unit="notional",
    )

distributions = distributions()
print(distributions.head())
print(distributions.columns)
all_columns = ["timepoint_id", "sample_id", 'infected_observable_state',
       'exposed_observable_state', 'hospitalized_observable_state',
       'dead_observable_state']
all_columns = ["timepoint_id", "sample_id", 'infected_observable_state']

distributions = distributions[all_columns]

   timepoint_id  sample_id  persistent_beta_c_param  persistent_kappa_param  \
0             0          0                 0.128003                0.125241   
1             1          0                 0.128003                0.125241   
2             2          0                 0.128003                0.125241   
3             3          0                 0.128003                0.125241   
4             4          0                 0.128003                0.125241   

   persistent_gamma_param  persistent_hosp_param  persistent_death_hosp_param  \
0                0.361102                0.17915                     0.074241   
1                0.361102                0.17915                     0.074241   
2                0.361102                0.17915                     0.074241   
3                0.361102                0.17915                     0.074241   
4                0.361102                0.17915                     0.074241   

   persistent_I0_param   D_state    E_

In [116]:
# def traces(distributions):
#     return (
#         distributions[distributions["sample_id"] == 0]
#         .set_index("timepoint_notional")[["dead_observable_state", "I_state"]]
#         .rename(
#             columns={
#                 "dead_observable_state": "dead_exemplar",
#                 "I_state": "I_exemplar",
#             }
#         )
#     )
# traces = traces(distributions)
# traces

In [117]:
%matplotlib inline

_output_root = "output_images/"

def save_result(data, name, ref_ext):
    """Save new reference files"""
    # _output_root.mkdir(parents=True, exist_ok=True)

    mode = "w" if ref_ext == "svg" else "wb"
    with open(os.path.join(_output_root, f"{name}.{ref_ext}"), mode) as f:
        f.write(data)
        
#select_by "mean", "var", "granger"

traces = pyciemss.visuals.trajectories.select_traces(distributions, select_by = "mean")
traces["timepoint_id"] = traces.index
traces = traces.rename(columns={c: c+'_mean' for c in traces.columns if c not in ["timepoint_id"]})
traces_var = pyciemss.visuals.trajectories.select_traces(distributions, select_by = "var")
traces_var["timepoint_id"] = traces.index
traces_var = traces_var.rename(columns={c: c+'_var' for c in traces_var.columns if c not in ["timepoint_id"]})
traces_granger = pyciemss.visuals.trajectories.select_traces(distributions, select_by = "granger")
traces_granger["timepoint_id"] = traces.index
traces_granger = traces_granger.rename(columns={c: c+'_granger' for c in traces_granger.columns if c not in ["timepoint_id"]})

new_df = pd.merge(traces, traces_var,  how='left', on=["timepoint_id"])
traces = pd.merge(new_df, traces_granger,  how='left', on=["timepoint_id"])


Granger Causality
number of lags (no zero) 10
ssr based F test:         F=2.8761  , p=0.0047  , df_denom=68, df_num=10
ssr based chi2 test:   chi2=37.6427 , p=0.0000  , df=10
likelihood ratio test: chi2=31.3932 , p=0.0005  , df=10
parameter F test:         F=3.3620  , p=0.0027  , df_denom=68, df_num=8

Granger Causality
number of lags (no zero) 10
ssr based F test:         F=19.0399 , p=0.0000  , df_denom=68, df_num=10
ssr based chi2 test:   chi2=249.1992, p=0.0000  , df=10
likelihood ratio test: chi2=118.8149, p=0.0000  , df=10
parameter F test:         F=19.0428 , p=0.0000  , df_denom=68, df_num=10

Granger Causality
number of lags (no zero) 10
ssr based F test:         F=2.8251  , p=0.0054  , df_denom=68, df_num=10
ssr based chi2 test:   chi2=36.9749 , p=0.0001  , df=10
likelihood ratio test: chi2=30.9227 , p=0.0006  , df=10
parameter F test:         F=3.3404  , p=0.0028  , df_denom=68, df_num=8

Granger Causality
number of lags (no zero) 10
ssr based F test:         F=72.2333 , p=

/Users/oost464/opt/anaconda3/envs/pyciems310/lib/python3.10/site-packages/statsmodels/base/model.py:1896: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 10, but rank is 8
  warnings.warn('covariance of constraints does not have full '
/Users/oost464/opt/anaconda3/envs/pyciems310/lib/python3.10/site-packages/statsmodels/base/model.py:1896: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 10, but rank is 8
  warnings.warn('covariance of constraints does not have full '
/Users/oost464/opt/anaconda3/envs/pyciems310/lib/python3.10/site-packages/statsmodels/base/model.py:1896: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 10, but rank is 9
  warnings.warn('covariance of constraints does not have full '
/Users/oost464/opt/anaconda3/envs/pyciems310/lib/python3.10/site-packages/statsmodels/base/model.py:1896: ValueWarning: covariance of constraints does not

In [118]:
schema = plots.trajectories(distributions, traces = traces,  keep=".*_state")

plots.save_schema(schema, "_schema.json")

plots.ipy_display(schema, format="interactive")
image = plots.ipy_display(schema, format="PNG").data
save_result(image, "traj_infected_observable", "png")


In [119]:
#'timepoint_id', 'sample_id'
sin_distribution = pd.read_csv("paper_sin_05/paper_sin_05_new_df.csv")
sin_distribution = sin_distribution.rename(
            columns={
                'Unnamed: 0.1': "timepoint_id",
                'Unnamed: 0': "100",
            }
        )
sin_distribution = sin_distribution.melt(id_vars=["timepoint_id"], 
        var_name="sample_id", 
        value_name="same_stat_state")


traces = pyciemss.visuals.trajectories.select_traces(sin_distribution, select_by = "mean")
traces['traces_var'] = pyciemss.visuals.trajectories.select_traces(sin_distribution, select_by = "var")["same_stat_state"]
traces['traces_granger'] = pyciemss.visuals.trajectories.select_traces(sin_distribution, select_by = "granger")["same_stat_state"]
schema = plots.trajectories(sin_distribution, traces = traces)
print(traces_var)
plots.save_schema(schema, "_schema.json")

plots.ipy_display(schema, format="interactive")
image = plots.ipy_display(schema, format="PNG").data
save_result(image, "paper_sin_05", "png")


Granger Causality
number of lags (no zero) 10
ssr based F test:         F=1.5077  , p=0.1554  , df_denom=69, df_num=10
ssr based chi2 test:   chi2=19.6658 , p=0.0326  , df=10
likelihood ratio test: chi2=17.7865 , p=0.0587  , df=10
parameter F test:         F=1.5077  , p=0.1554  , df_denom=69, df_num=10

Granger Causality
number of lags (no zero) 10
ssr based F test:         F=1.7838  , p=0.0800  , df_denom=69, df_num=10
ssr based chi2 test:   chi2=23.2664 , p=0.0098  , df=10
likelihood ratio test: chi2=20.6939 , p=0.0233  , df=10
parameter F test:         F=1.7838  , p=0.0800  , df_denom=69, df_num=10

Granger Causality
number of lags (no zero) 10
ssr based F test:         F=0.5552  , p=0.8443  , df_denom=69, df_num=10
ssr based chi2 test:   chi2=7.2420  , p=0.7024  , df=10
likelihood ratio test: chi2=6.9653  , p=0.7287  , df=10
parameter F test:         F=0.5552  , p=0.8443  , df_denom=69, df_num=10

Granger Causality
number of lags (no zero) 10
ssr based F test:         F=1.1772  , 

In [120]:
#'timepoint_id', 'sample_id'
sin_distribution = pd.read_csv("paper_sin/paper_sin_new_df.csv")
sin_distribution = sin_distribution.rename(
            columns={
                'Unnamed: 0': "timepoint_id",
            }
        )
sin_distribution = sin_distribution.melt(id_vars=["timepoint_id"], 
        var_name="sample_id", 
        value_name="same_stat_state")


traces = pyciemss.visuals.trajectories.select_traces(sin_distribution, select_by = "mean")
traces = traces.rename(
            columns={
                'same_stat_state': "traces_mean",
            }
        )
traces['traces_var'] = pyciemss.visuals.trajectories.select_traces(sin_distribution, select_by = "var")["same_stat_state"]
traces['traces_granger'] = pyciemss.visuals.trajectories.select_traces(sin_distribution, select_by = "granger")["same_stat_state"]
schema = plots.trajectories(sin_distribution, traces = traces)
print(traces_var)
plots.save_schema(schema, "_schema.json")

plots.ipy_display(schema, format="interactive")
image = plots.ipy_display(schema, format="PNG").data
save_result(image, "paper_sin", "png")


Granger Causality
number of lags (no zero) 10
ssr based F test:         F=3.5378  , p=0.0008  , df_denom=69, df_num=10
ssr based chi2 test:   chi2=46.1456 , p=0.0000  , df=10
likelihood ratio test: chi2=37.2524 , p=0.0001  , df=10
parameter F test:         F=3.5378  , p=0.0008  , df_denom=69, df_num=10

Granger Causality
number of lags (no zero) 10
ssr based F test:         F=4.3569  , p=0.0001  , df_denom=69, df_num=10
ssr based chi2 test:   chi2=56.8291 , p=0.0000  , df=10
likelihood ratio test: chi2=44.0513 , p=0.0000  , df=10
parameter F test:         F=4.3569  , p=0.0001  , df_denom=69, df_num=10

Granger Causality
number of lags (no zero) 10
ssr based F test:         F=3.1382  , p=0.0023  , df_denom=69, df_num=10
ssr based chi2 test:   chi2=40.9324 , p=0.0000  , df=10
likelihood ratio test: chi2=33.7385 , p=0.0002  , df=10
parameter F test:         F=3.1382  , p=0.0023  , df_denom=69, df_num=10

Granger Causality
number of lags (no zero) 10
ssr based F test:         F=2.3890  , 

In [121]:
#'timepoint_id', 'sample_id'
sin_distribution = pd.read_csv("paper_sin/paper_sin_old_df.csv")
sin_distribution = sin_distribution.rename(
            columns={
                'Unnamed: 0': "timepoint_id",
            }
        )
sin_distribution = sin_distribution.melt(id_vars=["timepoint_id"], 
        var_name="sample_id", 
        value_name="same_stat_state")


traces = pyciemss.visuals.trajectories.select_traces(sin_distribution, select_by = "mean")
traces = traces.rename(
            columns={
                'same_stat_state': "traces_mean",
            }
        )
traces['traces_var'] = pyciemss.visuals.trajectories.select_traces(sin_distribution, select_by = "var")["same_stat_state"]
traces['traces_granger'] = pyciemss.visuals.trajectories.select_traces(sin_distribution, select_by = "granger")["same_stat_state"]
schema = plots.trajectories(sin_distribution, traces = traces)
print(traces_var)
plots.save_schema(schema, "_schema.json")

plots.ipy_display(schema, format="interactive")
image = plots.ipy_display(schema, format="PNG").data
save_result(image, "paper_start", "png")


Granger Causality
number of lags (no zero) 10
ssr based F test:         F=-7.8988 , p=1.0000  , df_denom=79, df_num=10
ssr based chi2 test:   chi2=-89.9865, p=1.0000  , df=10
likelihood ratio test: chi2=-792.3666, p=1.0000  , df=10
parameter F test:         F=9832.5499, p=0.0000  , df_denom=79, df_num=6

Granger Causality
number of lags (no zero) 10
ssr based F test:         F=-7.8986 , p=1.0000  , df_denom=79, df_num=10
ssr based chi2 test:   chi2=-89.9844, p=1.0000  , df=10
likelihood ratio test: chi2=-779.4979, p=1.0000  , df=10
parameter F test:         F=11157.7647, p=0.0000  , df_denom=79, df_num=6

Granger Causality
number of lags (no zero) 10
ssr based F test:         F=-7.1956 , p=1.0000  , df_denom=79, df_num=10
ssr based chi2 test:   chi2=-81.9749, p=1.0000  , df=10
likelihood ratio test: chi2=-217.5511, p=1.0000  , df=10
parameter F test:         F=5832969.5496, p=0.0000  , df_denom=79, df_num=6

Granger Causality
number of lags (no zero) 10
ssr based F test:         F=-7.

/Users/oost464/opt/anaconda3/envs/pyciems310/lib/python3.10/site-packages/statsmodels/base/model.py:1896: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 10, but rank is 6
  warnings.warn('covariance of constraints does not have full '
/Users/oost464/opt/anaconda3/envs/pyciems310/lib/python3.10/site-packages/statsmodels/base/model.py:1896: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 10, but rank is 6
  warnings.warn('covariance of constraints does not have full '
/Users/oost464/opt/anaconda3/envs/pyciems310/lib/python3.10/site-packages/statsmodels/base/model.py:1896: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 10, but rank is 6
  warnings.warn('covariance of constraints does not have full '
/Users/oost464/opt/anaconda3/envs/pyciems310/lib/python3.10/site-packages/statsmodels/base/model.py:1896: ValueWarning: covariance of constraints does not

ssr based F test:         F=-7.8784 , p=1.0000  , df_denom=79, df_num=10
ssr based chi2 test:   chi2=-89.7535, p=1.0000  , df=10
likelihood ratio test: chi2=-531.0184, p=1.0000  , df=10
parameter F test:         F=180680.8853, p=0.0000  , df_denom=79, df_num=6

Granger Causality
number of lags (no zero) 10
ssr based F test:         F=-7.8631 , p=1.0000  , df_denom=79, df_num=10
ssr based chi2 test:   chi2=-89.5799, p=1.0000  , df=10
likelihood ratio test: chi2=-483.0319, p=1.0000  , df=10
parameter F test:         F=305790.2533, p=0.0000  , df_denom=79, df_num=6

Granger Causality
number of lags (no zero) 10
ssr based F test:         F=-7.8977 , p=1.0000  , df_denom=79, df_num=10
ssr based chi2 test:   chi2=-89.9736, p=1.0000  , df=10
likelihood ratio test: chi2=-732.1803, p=1.0000  , df=10
parameter F test:         F=18989.1084, p=0.0000  , df_denom=79, df_num=6

Granger Causality
number of lags (no zero) 10
ssr based F test:         F=-7.8779 , p=1.0000  , df_denom=79, df_num=10
ssr 

/Users/oost464/opt/anaconda3/envs/pyciems310/lib/python3.10/site-packages/statsmodels/base/model.py:1896: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 10, but rank is 6
  warnings.warn('covariance of constraints does not have full '
/Users/oost464/opt/anaconda3/envs/pyciems310/lib/python3.10/site-packages/statsmodels/base/model.py:1896: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 10, but rank is 6
  warnings.warn('covariance of constraints does not have full '
/Users/oost464/opt/anaconda3/envs/pyciems310/lib/python3.10/site-packages/statsmodels/base/model.py:1896: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 10, but rank is 6
  warnings.warn('covariance of constraints does not have full '
/Users/oost464/opt/anaconda3/envs/pyciems310/lib/python3.10/site-packages/statsmodels/base/model.py:1896: ValueWarning: covariance of constraints does not

ssr based F test:         F=-7.8872 , p=1.0000  , df_denom=79, df_num=10
ssr based chi2 test:   chi2=-89.8541, p=1.0000  , df=10
likelihood ratio test: chi2=-578.2452, p=1.0000  , df=10
parameter F test:         F=105861.9438, p=0.0000  , df_denom=79, df_num=6

Granger Causality
number of lags (no zero) 10
ssr based F test:         F=-3.7004 , p=1.0000  , df_denom=79, df_num=10
ssr based chi2 test:   chi2=-42.1567, p=1.0000  , df=10
likelihood ratio test: chi2=-56.8690, p=1.0000  , df=10
parameter F test:         F=35295813.9609, p=0.0000  , df_denom=79, df_num=6

Granger Causality
number of lags (no zero) 10
ssr based F test:         F=-7.8970 , p=1.0000  , df_denom=79, df_num=10
ssr based chi2 test:   chi2=-89.9661, p=1.0000  , df=10
likelihood ratio test: chi2=-709.6945, p=1.0000  , df=10
parameter F test:         F=24424.8915, p=0.0000  , df_denom=79, df_num=6

Granger Causality
number of lags (no zero) 10
ssr based F test:         F=-7.8985 , p=1.0000  , df_denom=79, df_num=10
ssr

/Users/oost464/opt/anaconda3/envs/pyciems310/lib/python3.10/site-packages/statsmodels/base/model.py:1896: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 10, but rank is 6
  warnings.warn('covariance of constraints does not have full '
/Users/oost464/opt/anaconda3/envs/pyciems310/lib/python3.10/site-packages/statsmodels/base/model.py:1896: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 10, but rank is 6
  warnings.warn('covariance of constraints does not have full '
/Users/oost464/opt/anaconda3/envs/pyciems310/lib/python3.10/site-packages/statsmodels/base/model.py:1896: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 10, but rank is 6
  warnings.warn('covariance of constraints does not have full '
/Users/oost464/opt/anaconda3/envs/pyciems310/lib/python3.10/site-packages/statsmodels/base/model.py:1896: ValueWarning: covariance of constraints does not

trajectory  infected_observable_state_var  timepoint_id
timepoint                                              
0                               11.969685             0
1                               15.828713             1
2                               18.445007             2
3                               20.906946             3
4                               23.540363             4
...                                   ...           ...
94                          348138.093750            94
95                          325088.843750            95
96                          302895.062500            96
97                          281833.750000            97
98                          261997.953125            98

[99 rows x 2 columns]

